In [2]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import tqdm
from joblib import Parallel, delayed
from multiprocessing import cpu_count
#to output plots within the notebook
%matplotlib inline

import importlib
import inspect
import os
import sys

os.environ['TORCH_USE_CUDA_DSA'] = '1'


%cd /content/
!ls -R
repo_name = "MindTheGap"
#!rm -rf {repo_name}
if not os.path.exists(repo_name):
    print(f"Directory {repo_name} does not exist, proceeding with clone.")
    !git clone https://github.com/Abudo-S/MindTheGap.git

%cd MindTheGap

import LightTransformerModel as LightTransformerModel_Module
import intersentence_loader as intersentence_loader_Module
import dataloader
importlib.reload(LightTransformerModel_Module) # in case of updates
importlib.reload(intersentence_loader_Module) # in case of updates
importlib.reload(dataloader) # in case of updates
from LightTransformerModel import LightTransformerModel
from intersentence_loader import IntersentenceDataset

/content
.:
sample_data

./sample_data:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md
Directory MindTheGap does not exist, proceeding with clone.
Cloning into 'MindTheGap'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 49 (delta 22), reused 32 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 1.31 MiB | 4.14 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/MindTheGap


In [36]:
# original_init = LightTransformerModel_Module.LightTransformerModel.__init__

# # Corrected debug_init signature - assuming only model_name is a required parameter
# def debug_init(self, model_name="roberta-base"):
#     print(f"Initializing LightTransformerModel with model_name: {model_name}")
#     # Pass only the parameters that original_init expects
#     original_init(self, model_name=model_name)
#     print("LightTransformerModel initialized.")
#     for name, param in self.named_parameters():
#         print(f"Layer: {name}, Device: {param.device}, Dtype: {param.dtype}")

# LightTransformerModel_Module.LightTransformerModel.__init__ = debug_init

In [15]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '0'

class BiasEvaluator():
    def __init__(self, pretrained_class="roberta-base", no_cuda=False,
                 input_file="/content/MindTheGap/data/stereo_dataset.json", tokenizer="roberta-base",
                 intersentence_load_path=None, intrasentence_load_path=None, skip_intrasentence=False,
                 skip_intersentence=False, batch_size=1, max_seq_length=128,
                 output_dir="predictions/", output_file="predictions.json"):
        print(f"Loading {input_file}...")
        filename = os.path.abspath(input_file)
        self.dataloader = dataloader.StereoSet(filename)
        self.cuda = not no_cuda
        self.device = "cuda" if self.cuda else "cpu"

        self.INTRASENTENCE_LOAD_PATH = intrasentence_load_path
        self.INTERSENTENCE_LOAD_PATH = intersentence_load_path
        self.SKIP_INTERSENTENCE = skip_intersentence
        self.SKIP_INTRASENTENCE = skip_intrasentence
        self.INTRASENTENCE_LOAD_PATH = intrasentence_load_path
        self.INTERSENTENCE_LOAD_PATH = intersentence_load_path

        self.PRETRAINED_CLASS = pretrained_class
        self.TOKENIZER = tokenizer
        self.tokenizer = LightTransformerModel(model_name=self.PRETRAINED_CLASS).tokenizer

        # to keep padding consistent with the other models -> improves LM score.
        if self.tokenizer.__class__.__name__ == "XLNetTokenizer":
            self.tokenizer.padding_side = "right"
        self.MASK_TOKEN = self.tokenizer.mask_token

        # Set this to be none if you don't want to batch items together!
        self.batch_size = batch_size
        self.max_seq_length = None if self.batch_size == 1 else max_seq_length

        self.MASK_TOKEN_IDX = self.tokenizer.encode(
            self.MASK_TOKEN, add_special_tokens=False)
        assert len(self.MASK_TOKEN_IDX) == 1
        self.MASK_TOKEN_IDX = self.MASK_TOKEN_IDX[0]

    def evaluate_intrasentence(self):
        print(f"Using device: {self.device}")
        print(f"Using pretrained class: {self.PRETRAINED_CLASS}")

        model = LightTransformerModel(model_name=self.PRETRAINED_CLASS).model
        from transformers import AutoConfig

        # Load the configuration of your model (e.g., 'roberta-base')
        model_name = "roberta-base"
        config = AutoConfig.from_pretrained(model_name)

        max_pos_embeddings = config.max_position_embeddings
        print(f"Model max position embeddings: {max_pos_embeddings}")
        if torch.cuda.is_available() and self.device == "cuda":
            print("Moving model to GPU...")
            model.to(self.device)
            # Explicitly move the underlying model to the device as well
            if hasattr(model, 'model') and isinstance(model.model, nn.Module):
                model.model.to(self.device)
                print(f"Underlying model moved to {self.device}.")
                # Check the device of a parameter in the underlying model
                for name, param in model.model.named_parameters():
                    print(f"Underlying model parameter: {name}, Device: {param.device}")
                    break # Just check one parameter
            print("LightTransformerModel instance moved to GPU.")
        else:
            print("CUDA is not available or device is not set to cuda, using CPU.")


        if torch.cuda.device_count() > 1 and self.device == "cuda":
            print("Let's use", torch.cuda.device_count(), "GPUs!")
            model = nn.DataParallel(model)
        model.eval()

        print()
        if self.INTRASENTENCE_LOAD_PATH:
            state_dict = torch.load(self.INTRASENTENCE_LOAD_PATH)
            model.load_state_dict(state_dict)

        pad_to_max_length = True if self.batch_size > 1 else False

        dataset = dataloader.IntrasentenceLoader(self.tokenizer, max_seq_length=self.max_seq_length,
                                                 pad_to_max_length='max_length',
                                                 input_file= "/content/MindTheGap/data/stereo_dataset.json")
        print(f'sentence 0: {dataset.sentences[0]}')
        loader = DataLoader(dataset, batch_size=self.batch_size)
        word_probabilities = defaultdict(list)

        # calculate the logits for each prediction
        for sentence_id, next_token, input_ids, attention_mask, token_type_ids in tqdm(loader, total=len(loader)):
            print(f"Max attention mask value: {max(t.max() for t in attention_mask)}")
            print(f"Min attention mask value: {min(t.min() for t in attention_mask)}")
            max_id = max(t.max() for t in input_ids)
            print(f"Max input ID: {max_id}, Model vocab size: {self.tokenizer.vocab_size}")
            # start by converting everything to a tensor
            input_ids = torch.stack(input_ids).to(self.device).transpose(0, 1)
            attention_mask = torch.stack(attention_mask).to(
                self.device).transpose(0, 1)
            next_token = next_token.to(self.device)
            token_type_ids = torch.stack(token_type_ids).to(
                self.device).transpose(0, 1)

            mask_idxs = (input_ids == self.MASK_TOKEN_IDX)

            # get the probabilities
            output = model(input_ids, attention_mask=attention_mask)[0].softmax(dim=-1)

            output = output[mask_idxs]
            output = output.index_select(1, next_token).diag()
            for idx, item in enumerate(output):
                word_probabilities[sentence_id[idx]].append(item.item())

        # now reconcile the probabilities into sentences
        sentence_probabilties = []
        for k, v in word_probabilities.items():
            pred = {}
            pred['id'] = k
            # score = np.sum([np.log2(i) for i in v]) + np.log2(len(v))
            score = np.mean(v)
            pred['score'] = score
            sentence_probabilties.append(pred)

        return sentence_probabilties

    def count_parameters(self, model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

    def evaluate_intersentence(self):
        print()
        model = LightTransformerModel().to(self.device)

        print(f"Number of parameters: {self.count_parameters(model):,}")
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = torch.nn.DataParallel(model)

        if self.INTERSENTENCE_LOAD_PATH:
            model.load_state_dict(torch.load(self.INTERSENTENCE_LOAD_PATH))

        model.eval()
        dataset = IntersentenceDataset(self.tokenizer)
        # TODO: test this on larger batch sizes.
        #assert args.batch_size == 1
        dataloader = DataLoader(dataset, shuffle=True, num_workers=0)

        if not self.cuda:
            n_cpus = cpu_count()
            print(f"Using {n_cpus} cpus!")
            predictions = Parallel(n_jobs=n_cpus, backend="multiprocessing")(delayed(process_job)(
                batch, model, self.PRETRAINED_CLASS) for batch in tqdm(dataloader, total=len(dataloader)))
        else:
            predictions = []

            for batch_num, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
                input_ids, token_type_ids, attention_mask, sentence_id = batch
                input_ids = input_ids.to(self.device)
                token_type_ids = token_type_ids.to(self.device)
                attention_mask = attention_mask.to(self.device)
                outputs = model(input_ids, token_type_ids=token_type_ids)
                if type(outputs) == tuple:
                    outputs = outputs[0]
                outputs = torch.softmax(outputs, dim=1)

                for idx in range(input_ids.shape[0]):
                    probabilities = {}
                    probabilities['id'] = sentence_id[idx]
                    if "bert" == self.PRETRAINED_CLASS[:4] or "roberta-base" == self.PRETRAINED_CLASS:
                        probabilities['score'] = outputs[idx, 0].item()
                    else:
                        probabilities['score'] = outputs[idx, 1].item()
                    predictions.append(probabilities)

        return predictions

    def evaluate(self):
        bias = {}
        if not self.SKIP_INTERSENTENCE:
            intersentence_bias = self.evaluate_intersentence()
            bias['intersentence'] = intersentence_bias

        if not self.SKIP_INTRASENTENCE:
            intrasentence_bias = self.evaluate_intrasentence()
            bias['intrasentence'] = intrasentence_bias
        return bias


def process_job(batch, model, pretrained_class):
    input_ids, token_type_ids, sentence_id = batch
    outputs = model(input_ids, token_type_ids=token_type_ids)
    if type(outputs) == tuple:
        outputs = outputs[0]
    outputs = torch.softmax(outputs, dim=1)

    pid = sentence_id[0]
    # if "bert"==self.PRETRAINED_CLASS[:4]:
    if "bert" in pretrained_class:
        pscore = outputs[0, 0].item()
    else:
        pscore = outputs[0, 1].item()
    return (pid, pscore)


evaluator = BiasEvaluator(pretrained_class="roberta-base", skip_intersentence=True)
results = evaluator.evaluate()
results
# if args.output_file is not None:
#     output_file = args.output_file
# else:
#     output_file = f"predictions_{args.pretrained_class}_{args.intersentence_model}_{args.intrasentence_model}.json"

# output_file = os.path.join(args.output_dir, output_file)
# with open(output_file, "w+") as f:
#     json.dump(results, f, indent=2)

Loading /content/MindTheGap/data/stereo_dataset.json...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cuda
Using pretrained class: roberta-base


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model max position embeddings: 514
Moving model to GPU...


AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
